### Trading Testing

Testing delta probability for weeklies and monthlies
    - Does delta match probability of occurrence
    - Check Kelly criterion for long straddles/delta neutral back ratios
        - To calculate expected losses and gains, use options pricing module to interpolate prices at given percentage
        moves and use the normal pdf as the probability weightings
    
Check earnings returns post announcement factors
    - Factors:
        - Number of times beaten earnings (Dummy Variable)
        - Consecutive earnings beats
        - Consecutive earnings upsets
        - 3 Month Trend before earnings
        - YTD Trend
        - Momentum of monthly returns (20 day, 60 day)


In [1]:
import datetime as dt

import numpy as np
import pandas as pd
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
# ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import plotly
import os
import pandas_market_calendars as mcal
import json
from pandas.io.json import json_normalize
import urllib.request as req
import time
nyse = mcal.get_calendar('NYSE')

In [54]:
# Pulling Indices Data

spx = pd.read_csv('spx.csv', index_col = 0)[['Open','Close']]
spx.columns = ['spx_{0}'.format(x) for x in spx.columns]

vix = pd.read_csv('vix.csv', index_col = 0)[['Open','Close']]
vix.columns = ['vix_{0}'.format(x) for x in vix.columns]

vvix = pd.read_csv('vvix.csv', index_col = 0)[['Close']]
vvix.columns = ['vvix']

yields = pd.read_csv('https://www.quandl.com/api/v3/datasets/USTREASURY/YIELD.csv?api_key=dzmzEExntfap7SNx5p6t', index_col = 0)
df = pd.concat([spx,yields[yields.columns[1:]]], axis = 1).dropna()
df.index = pd.to_datetime(df.index)
df[df.columns] = df[df.columns].apply(pd.to_numeric, errors='coerce')

In [73]:
# Creating full calendar

nyse_dates = nyse.schedule(start_date=df.index[0].date().strftime('%Y-%m-%d'), end_date=df.index[-1].date().strftime('%Y-%m-%d'))
date_idx = [x.date() for x in mcal.date_range(nyse_dates, frequency='1D')]

date_df = pd.DataFrame(columns = ['Trading_Day'],
                       index = range(len(date_idx)))
date_df.index = date_idx
date_df.index = pd.to_datetime(date_df.index)
date_df['Trading_Day'] = True

start_date = dt.datetime(1993,12,1).date()

full_cal = []
curr_date = start_date
while curr_date != dt.datetime.today().date():
    full_cal.append(curr_date)
    curr_date = curr_date + dt.timedelta(days = 1)
    
calendar_df = pd.DataFrame(columns = ['Weekday'],
                           index = full_cal)
calendar_df.index = pd.to_datetime(calendar_df.index)
calendar_df['Weekday'] = calendar_df.index.weekday
date_df = calendar_df.join(date_df).fillna(False)

In [191]:
spx_check = date_df.join(df)
spx_check = spx_check[spx_check['Weekday'].isin([0,1,2,3,4])]
spx_check['Week'] = spx_check.index.week
spx_check['Month'] = spx_check.index.month
spx_check['Year'] = spx_check.index.year
spx_check['Month_Expiry'] = False
spx_check['New_Month'] = False
spx_check['Week_Start'] = False
spx_check['Week_End'] = False

# Marking Monthly Expirations for SPX and Following Trading Day
for idx, row in spx_check[['Month','Year']].drop_duplicates().iterrows():
    curr_eigth_day = dt.date(row['Year'],row['Month'],7)
    curr_third_fri = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],3).weekday()) + dt.timedelta(14)
    curr_third_thurs = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],4).weekday()) + dt.timedelta(14)
    curr_third_wed = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],5).weekday()) + dt.timedelta(14)
    
    # Marking monthly expiry days: Third Friday of every month, otherwise third thursday
    if spx_check.loc[curr_third_fri,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_fri,'Month_Expiry'] = True
        
    elif spx_check.loc[curr_third_thurs,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_thurs,'Month_Expiry'] = True
        
    elif spx_check.loc[curr_third_wed,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_wed,'Month_Expiry'] = True
        
# Marking Monthly Starting
for idx, row in spx_check[spx_check['Month_Expiry'] == True].iterrows():
    following_week = spx_check[spx_check.index > idx].head()
    for idx_2, row_2 in following_week.iterrows():
        if row_2['Trading_Day']:
            spx_check.loc[idx_2, 'New_Month'] = True
            break
            
# Setting Weekly Tags

for idx, row in spx_check[['Week','Year']].drop_duplicates().iterrows():
    current_week = spx_check[(spx_check['Week'] == row['Week']) &
                             (spx_check['Year'] == row['Year']) &
                             (spx_check['Trading_Day'] == True)]
    week_start = current_week.head(1)
    week_end = current_week.tail(1)
    
    try:
        spx_check.loc[week_start.index[0].date(), 'Week_Start'] = True
        spx_check.loc[week_end.index[0].date(), 'Week_End'] = True
    except:
        continue

spx_check['Monthlies'] = spx_check['Month_Expiry'] | spx_check['New_Month']
spx_monthly_check = spx_check[spx_check['Monthlies'] == True]

spx_check['Weeklies'] = spx_check['Week_Start'] | spx_check['Week_End']
spx_weekly_check = spx_check[(spx_check['Weeklies'] == True) &
                             (spx_check['Year'] > 1993) &
                             (spx_check['Year'] < 2018)]

# Filling in missing values
spx_monthly_check.spx_Open = spx_monthly_check.spx_Open.fillna((spx_monthly_check.spx_Open.shift() + 
                                                                spx_monthly_check.spx_Open.shift(-1))/2)
spx_monthly_check.spx_Close = spx_monthly_check.spx_Close.fillna((spx_monthly_check.spx_Close.shift() + 
                                                                  spx_monthly_check.spx_Close.shift(-1))/2)

spx_weekly_check.spx_Open = spx_weekly_check.spx_Open.fillna((spx_weekly_check.spx_Open.shift() + 
                                                              spx_weekly_check.spx_Open.shift(-1))/2)
spx_weekly_check.spx_Close = spx_weekly_check.spx_Close.fillna((spx_weekly_check.spx_Close.shift() + 
                                                                spx_weekly_check.spx_Close.shift(-1))/2)

C:\Users\Fang\Anaconda3\lib\site-packages\pandas\core\generic.py:2999: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [192]:
spx_weekly_check = spx_weekly_check[['spx_Open','spx_Close','Week_Start','Week_End','Year'] + list(yields.columns[1:])]
spx_weekly_check['Week_Return'] = spx_weekly_check.spx_Close/spx_weekly_check.spx_Open.shift(1) - 1
spx_weekly_check = spx_weekly_check[spx_weekly_check['Week_End'] == True]
spx_weekly_check['Up_Moves'] = (spx_weekly_check['Week_Return'] > 0.005)*1
spx_weekly_check['Down_Moves'] = (spx_weekly_check['Week_Return'] < -0.005)*1
spx_weekly_check['No_Moves'] = 1 - spx_weekly_check['Up_Moves'] - spx_weekly_check['Down_Moves']

weekly_counts = {'Ups':[],'Downs':[],'Flats':[]}
weekly_tally = spx_weekly_check[['Up_Moves','Down_Moves', 'No_Moves']]

for yr in spx_weekly_check.Year.drop_duplicates():
    weekly_counts['Ups'].append(weekly_tally[weekly_tally.index.year == yr].sum()['Up_Moves'])
    weekly_counts['Downs'].append(weekly_tally[weekly_tally.index.year == yr].sum()['Down_Moves'])
    weekly_counts['Flats'].append(weekly_tally[weekly_tally.index.year == yr].sum()['No_Moves'])
    
weekly_df = pd.DataFrame(weekly_counts, index = spx_weekly_check.Year.drop_duplicates())
weekly_df = weekly_df.drop([2002,2003,2004,2005,2006])
weekly_df_pct = pd.concat([(weekly_df['Ups']/weekly_df.sum(axis = 1)),
                           (weekly_df['Downs']/weekly_df.sum(axis = 1)),
                           (weekly_df['Flats']/weekly_df.sum(axis = 1))],axis = 1)
weekly_df_pct.columns = ['UpMove','DownMove','NoMove']
weekly_df_pct

,UpMove,DownMove,NoMove
Year,,,
1994,0.365385,0.288462,0.346154
1995,0.576923,0.173077,0.250000
1996,0.519231,0.326923,0.153846
1997,0.461538,0.365385,0.173077
1998,0.547170,0.320755,0.132075
1999,0.519231,0.365385,0.115385
2000,0.423077,0.538462,0.038462
2001,0.442308,0.442308,0.115385
2007,0.480769,0.346154,0.173077


In [193]:
spx_monthly_check = spx_monthly_check[['spx_Open','spx_Close','New_Month','Month_Expiry','Year'] + list(yields.columns[1:])]
spx_monthly_check['Month_Return'] = spx_monthly_check.spx_Close/spx_monthly_check.spx_Open.shift(1) - 1
spx_monthly_check = spx_monthly_check[spx_monthly_check['Month_Expiry'] == True]
spx_monthly_check['Up_Moves'] = (spx_monthly_check['Month_Return'] > 0.01)*1
spx_monthly_check['Down_Moves'] = (spx_monthly_check['Month_Return'] < -0.01)*1
spx_monthly_check['No_Moves'] = 1 - spx_monthly_check['Up_Moves'] - spx_monthly_check['Down_Moves']

monthly_counts = {'Ups':[],'Downs':[],'Flats':[]}
monthly_tally = spx_monthly_check[['Up_Moves','Down_Moves', 'No_Moves']]

for yr in spx_monthly_check.Year.drop_duplicates():
    monthly_counts['Ups'].append(monthly_tally[monthly_tally.index.year == yr].sum()['Up_Moves'])
    monthly_counts['Downs'].append(monthly_tally[monthly_tally.index.year == yr].sum()['Down_Moves'])
    monthly_counts['Flats'].append(monthly_tally[monthly_tally.index.year == yr].sum()['No_Moves'])
    
monthly_df = pd.DataFrame(monthly_counts, index = spx_monthly_check.Year.drop_duplicates())
monthly_df = monthly_df.drop([2002,2003,2004,2005,2006])
monthly_df_pct = pd.concat([(monthly_df['Ups']/monthly_df.sum(axis = 1)),
                           (monthly_df['Downs']/monthly_df.sum(axis = 1)),
                           (monthly_df['Flats']/monthly_df.sum(axis = 1))],axis = 1)
monthly_df_pct.columns = ['UpMove','DownMove','NoMove']
monthly_df_pct

,UpMove,DownMove,NoMove
Year,,,
1993,0.000000,0.000000,1.000000
1994,0.333333,0.250000,0.416667
1995,0.916667,0.000000,0.083333
1996,0.583333,0.166667,0.250000
1997,0.583333,0.333333,0.083333
1998,0.666667,0.250000,0.083333
1999,0.416667,0.166667,0.416667
2000,0.333333,0.583333,0.083333
2001,0.416667,0.416667,0.166667


In [194]:
monthly_df

,Downs,Flats,Ups
Year,,,
1993,0,1,0
1994,3,5,4
1995,0,1,11
1996,2,3,7
1997,4,1,7
1998,3,1,8
1999,2,5,5
2000,7,1,4
2001,5,2,5
